In [4]:
import os
import sys
import pathlib
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
import torch
import math
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision.transforms.functional import InterpolationMode
from torch.utils.data import DataLoader
from PIL import Image

import albumentations as A
from albumentations.pytorch import ToTensorV2

import models
import utils
import datasets
import config
utils.set_seed(42)

In [5]:
args = config.init_args(server=True, ipykernel=True)

args.excluded_heads []


In [9]:
model_info = {}
model_names = ['segformer', 'deeplabv3plus']

for model_name in model_names:
    model = models.get_network(model_name, 21, pretrained=True)
    num_parameters = sum(p.numel() for p in model.parameters())
    model_size = 0
    for p in model.parameters():
        model_size += p.element_size() * p.nelement()
    model_size = model_size / 1024 / 1024  # Bytes를 Megabytes로 변환
    model_info[model_name] = {
        'num_parameters': num_parameters,
        'model_size_MB': model_size
    }

for model_name, info in model_info.items():
    print(f"Model: {model_name}")
    print(f"  Number of parameters: {info['num_parameters']}")
    print(f"  Model size (MB): {info['model_size_MB']:.2f}")

Loading pretrained model...
Model: segformer
  Number of parameters: 3719541
  Model size (MB): 14.19
Model: deeplabv3plus
  Number of parameters: 22442597
  Model size (MB): 85.61


In [12]:
# make table for report
import pandas as pd
df = pd.DataFrame(model_info).T
df = df[['model_size_MB', 'num_parameters']]
df.columns = ['Model Size (MB)', 'Number of Parameters']
df.index.name = 'Model'
df.to_csv('model_size.csv')